In [189]:
import numpy as np
import random

In [190]:
random_list = [(random.randint(0, 31), random.randint(0, 15), random.randint(0, 5)) for _ in range(200)]

In [191]:
random_list

[(13, 10, 1),
 (3, 15, 1),
 (21, 5, 0),
 (0, 0, 1),
 (21, 4, 2),
 (24, 2, 5),
 (12, 14, 0),
 (11, 2, 3),
 (7, 4, 5),
 (5, 15, 4),
 (22, 3, 0),
 (31, 10, 5),
 (2, 4, 0),
 (15, 3, 2),
 (24, 13, 4),
 (27, 15, 0),
 (5, 10, 3),
 (23, 5, 3),
 (29, 4, 4),
 (11, 1, 1),
 (15, 0, 2),
 (17, 1, 0),
 (9, 6, 5),
 (11, 5, 2),
 (13, 3, 1),
 (18, 5, 5),
 (2, 12, 5),
 (2, 3, 2),
 (7, 12, 2),
 (24, 12, 3),
 (24, 4, 2),
 (30, 0, 1),
 (7, 1, 2),
 (1, 7, 4),
 (24, 7, 4),
 (24, 9, 2),
 (5, 7, 5),
 (0, 3, 3),
 (2, 1, 0),
 (1, 3, 0),
 (7, 8, 5),
 (14, 6, 0),
 (2, 8, 2),
 (12, 4, 3),
 (24, 5, 0),
 (20, 5, 2),
 (13, 6, 2),
 (5, 13, 0),
 (14, 9, 2),
 (11, 13, 0),
 (12, 11, 0),
 (10, 0, 2),
 (28, 0, 4),
 (23, 13, 2),
 (23, 4, 4),
 (7, 0, 1),
 (3, 0, 2),
 (1, 5, 1),
 (20, 5, 2),
 (8, 6, 3),
 (31, 14, 3),
 (21, 0, 4),
 (8, 5, 4),
 (22, 8, 4),
 (28, 3, 5),
 (24, 8, 0),
 (13, 7, 0),
 (13, 14, 1),
 (30, 3, 5),
 (17, 6, 4),
 (15, 0, 4),
 (23, 11, 0),
 (1, 4, 5),
 (10, 9, 5),
 (8, 3, 4),
 (21, 12, 5),
 (1, 11, 0),
 (19, 

In [192]:
M = sc.parallelize(random_list)

In [234]:
data = sc.textFile("ml-25m/ratings.csv")
header = data.first() #extract header
data = data.filter(lambda row: row != header)
M = data.map(
  lambda l: l.split(',')
).map(
  lambda l: (int(l[0]), int(l[1]), float(l[2]))
)


In [193]:
M.collect()

[(13, 10, 1),
 (3, 15, 1),
 (21, 5, 0),
 (0, 0, 1),
 (21, 4, 2),
 (24, 2, 5),
 (12, 14, 0),
 (11, 2, 3),
 (7, 4, 5),
 (5, 15, 4),
 (22, 3, 0),
 (31, 10, 5),
 (2, 4, 0),
 (15, 3, 2),
 (24, 13, 4),
 (27, 15, 0),
 (5, 10, 3),
 (23, 5, 3),
 (29, 4, 4),
 (11, 1, 1),
 (15, 0, 2),
 (17, 1, 0),
 (9, 6, 5),
 (11, 5, 2),
 (13, 3, 1),
 (18, 5, 5),
 (2, 12, 5),
 (2, 3, 2),
 (7, 12, 2),
 (24, 12, 3),
 (24, 4, 2),
 (30, 0, 1),
 (7, 1, 2),
 (1, 7, 4),
 (24, 7, 4),
 (24, 9, 2),
 (5, 7, 5),
 (0, 3, 3),
 (2, 1, 0),
 (1, 3, 0),
 (7, 8, 5),
 (14, 6, 0),
 (2, 8, 2),
 (12, 4, 3),
 (24, 5, 0),
 (20, 5, 2),
 (13, 6, 2),
 (5, 13, 0),
 (14, 9, 2),
 (11, 13, 0),
 (12, 11, 0),
 (10, 0, 2),
 (28, 0, 4),
 (23, 13, 2),
 (23, 4, 4),
 (7, 0, 1),
 (3, 0, 2),
 (1, 5, 1),
 (20, 5, 2),
 (8, 6, 3),
 (31, 14, 3),
 (21, 0, 4),
 (8, 5, 4),
 (22, 8, 4),
 (28, 3, 5),
 (24, 8, 0),
 (13, 7, 0),
 (13, 14, 1),
 (30, 3, 5),
 (17, 6, 4),
 (15, 0, 4),
 (23, 11, 0),
 (1, 4, 5),
 (10, 9, 5),
 (8, 3, 4),
 (21, 12, 5),
 (1, 11, 0),
 (19, 

In [235]:
numWorkers = 4

In [236]:
def assignBlockIndex (index, numData, numWorkers):
    blockSize = numData/numWorkers
    if(numData % numWorkers != 0): blockSize = blockSize + 1
    return int(np.floor(index/np.ceil(blockSize)))+1


In [237]:
numFactors = 10
numRows = M.max(lambda x : x[0])[0] + 1
numCols = M.max(lambda x : x[1])[1] + 1
avgRating = M.map(lambda x: x[2]).mean()

#compute the scaling factor that the randomly initialized W and H matrices need to be scaled by so that dot(W_0,H_0) results in values that are similar to the average ratings
scaleRating = np.sqrt(avgRating / numFactors)

tau = 100


In [238]:
numRows

162542

In [ ]:
Mblocked = M.keyBy(lambda x: assignBlockIndex(x[0], numRows, numWorkers)).partitionBy(numWorkers)
Mblocked.collect()

In [ ]:
W = M.map(lambda x: tuple([int(x[0]),1])).reduceByKey(lambda x,y : x+y).map(lambda x: tuple([x[0],tuple([x[1],scaleRating*np.random.rand(1, numFactors).astype(np.float32)])])).persist()
H = M.map(lambda x: tuple([int(x[1]),1])).reduceByKey(lambda x,y : x+y).map(lambda x: tuple([x[0],tuple([x[1],scaleRating*np.random.rand(numFactors,1).astype(np.float32)])])).persist()


Iterater turn

In [ ]:
it = 1
beta = 0.1
mse = sc.accumulator(0.0)
nUpdates = sc.accumulator(0.0)
#broadcast the stepsize
#stepSize = sc.broadcast(np.power(tau + it, -beta))
stepSize = sc.broadcast(0.5)
#generate random strata
lam = sc.broadcast(0.1)

perms = np.random.permutation(numWorkers)+1
perms

In [ ]:
perms[0]

In [ ]:
perms_dict = {i: val for i, val in enumerate(perms)}
perms_dict

In [ ]:
rev_perms=list(i+1 for i in (dict(sorted(perms_dict.items(), key=lambda item: item[1]))).keys())
rev_perms

In [ ]:
Mfilt = Mblocked.filter(lambda x: perms[x[0]-1]==assignBlockIndex(x[1][1],numCols,numWorkers)).persist()

In [ ]:
Mfilt.collect()

In [ ]:
Mfilt.count()

In [ ]:
Hblocked = H.keyBy(lambda x: rev_perms[assignBlockIndex(x[0], numCols, numWorkers)-1])
Wblocked = W.keyBy(lambda x: assignBlockIndex(x[0], numRows, numWorkers))

In [ ]:
Wblocked.collect()

In [ ]:
Wblocked.map(lambda x: x[1][0]).count()

In [ ]:
groupRDD = Mfilt.groupWith(Hblocked, Wblocked).partitionBy(numWorkers)

In [ ]:
groupRDD.collect()

In [ ]:
for i in groupRDD.collect()[1][1][0]:
    print(i)
print("---------H block -----------")
for i in groupRDD.collect()[1][1][1]:
    print(i)
print("---------W block -----------")
for i in groupRDD.collect()[1][1][2]:
    print(i)

In [ ]:
def SGD(keyed_iterable, stepSize, numFactors,lam, mse, nUpdates, scaleRating):
    for i in keyed_iterable:
        Miter = i[1][0]
        Hiter = i[1][1]
        Witer = i[1][2]
    
    Wdict = {}
    Hdict = {}
    
    #iterate through H and W and create dictionary of elements
    for h in Hiter:
        Hdict[h[0]] = h[1]
    
    for w in Witer:
        Wdict[w[0]] = w[1]
    print(Hdict)
    #iterate through entries of M and compute L2-loss
    for m in Miter:
        (i,j,rat) = m
        #if i not in Wdict:
        #    Wdict[i] = tuple([1,scaleRating*np.random.rand(1,numFactors).astype(np.float32)])
        #if j not in Hdict:
        #    Hdict[j] = tuple([1,scaleRating*np.random.rand(numFactors,1).astype(np.float32)])

        (Nw, Wprev) = Wdict[i]
        (Nh, Hprev) = Hdict[j]
        error = rat - Wprev.dot(Hprev)
        print(error)
        delta = -2*error
        mse += error**2

        Wnew = Wprev - stepSize.value*(-2*error*Hprev.T + (2.0*lam.value*Wprev))
        Hnew = Hprev - stepSize.value*(-2*error*Wprev.T + (2.0*lam.value*Hprev))

        nUpdates += 1

        Wdict[i] = tuple([Nw, Wnew])
        Hdict[j] = tuple([Nh, Hnew])
    return (tuple(['W',Wdict.items()]), tuple(['H',Hdict.items()]))

In [ ]:
def SGD(keyed_iterable, stepSize, numFactors,lam, mse, nUpdates, scaleRating):
    iterlist = (next(keyed_iterable))
    Miter = iterlist[1][0]
    Hiter = iterlist[1][1]
    Witer = iterlist[1][2]
    
    Wdict = {}
    Hdict = {}
    
    Wout = {}
    Hout = {}
    
    #iterate through H and W and create dictionary of elements
    for h in Hiter:
        Hdict[h[0]] = h[1]
    
    for w in Witer:
        Wdict[w[0]] = w[1]
    #iterate through entries of M and compute L2-loss
    for m in Miter:
        (i,j,rat) = m
        if i not in Wdict:
            Wdict[i] = tuple([i,scaleRating*np.random.rand(1,numFactors).astype(np.float32)])
        if j not in Hdict:
            Hdict[j] = tuple([j,scaleRating*np.random.rand(numFactors,1).astype(np.float32)])

        (Nw, Wprev) = Wdict[i]
        (Nh, Hprev) = Hdict[j]
        delta = -2*(rat - Wprev.dot(Hprev))
        mse += (rat - Wprev.dot(Hprev))**2

        Wnew = Wprev - stepSize.value*(delta*Hprev.T + (2.0*lam.value/Nh)*Wprev)
        Hnew = Hprev - stepSize.value*(delta*Wprev.T + (2.0*lam.value/Nw)*Hprev)
        nUpdates += 1

        Wout[i] = tuple([Nw, Wnew])
        Hout[j] = tuple([Nh, Hnew])
        
    return (tuple(['W',Wdict.items()]), tuple(['H',Hdict.items()]))

In [ ]:
WH = groupRDD.mapPartitions(lambda x: SGD(x, stepSize, numFactors,lam, mse, nUpdates,scaleRating))

In [ ]:
WH.collect()

In [ ]:
W.unpersist()
H.unpersist()

In [ ]:
W = WH.filter(lambda x: x[0]=='W').flatMap(lambda x: x[1]).persist()
H = WH.filter(lambda x: x[0]=='H').flatMap(lambda x: x[1]).persist()


In [ ]:
W.collect()

In [ ]:
M.filter(lambda x: x[0]==31).collect()

In [ ]:
w_arr = np.array(W.filter(lambda x: x[0]==31).map(lambda x: x[1][1]).collect())
h_arr = np.array(H.filter(lambda x: x[0]==15).map(lambda x: x[1][1]).collect())


In [ ]:
w_arr.dot(h_arr)

In [ ]:
mse = sc.accumulator(0.0)
nUpdates = sc.accumulator(0)

for it in range(10):
    beta = 0.1
    #broadcast the stepsize
    stepSize = sc.broadcast(np.power(tau + it, -beta))
    #generate random strata
    lam = sc.broadcast(0.1)

    perms = np.random.permutation(numWorkers)+1
    perms_dict = {i: val for i, val in enumerate(perms)}
    rev_perms=list(i+1 for i in (dict(sorted(perms_dict.items(), key=lambda item: item[1]))).keys())
    Mfilt = Mblocked.filter(lambda x: perms[x[0]-1]==assignBlockIndex(x[1][1],numCols,numWorkers)).persist()
    Hblocked = H.keyBy(lambda x: rev_perms[assignBlockIndex(x[0], numCols, numWorkers)-1])
    Wblocked = W.keyBy(lambda x: assignBlockIndex(x[0], numRows, numWorkers))
    groupRDD = Mfilt.groupWith(Hblocked, Wblocked).partitionBy(numWorkers)
    WH = groupRDD.mapPartitions(lambda x: SGD(x, stepSize, numFactors,lam, mse, nUpdates,scaleRating))
    W.unpersist()
    H.unpersist()
    W = WH.filter(lambda x: x[0]=='W').flatMap(lambda x: x[1]).persist()
    H = WH.filter(lambda x: x[0]=='H').flatMap(lambda x: x[1]).persist()
    Wvec = W.collect()
    Hvec = H.collect()
    print("MSE/update for {}-th iteration is: {}/{} ".format(it, mse.value, nUpdates.value))
    print("RMSE: {}".format((mse.value[0][0]/nUpdates.value)))


In [ ]:
def predict(userId, itemId):
    w_user = W.filter(lambda x: x[0] == 19).map(lambda x: x[1][1]).collect()
    h_item = H.filter(lambda x: x[0] == 8).map(lambda x: x[1][1]).collect()
    pred_rating = np.array(w_user).dot(h_item)
    

In [ ]:
w_26 = W.filter(lambda x: x[0] == 31).map(lambda x: x[1][1]).collect()
h_8 = H.filter(lambda x: x[0] == 5).map(lambda x: x[1][1]).collect()

In [ ]:
w_26

In [ ]:
np.array(w_26).dot(np.array(h_8))

In [ ]:
M.filter(lambda x: x[0]==31).collect()